In [31]:
import mysql.connector
import numpy as np
import pandas as pd
import requests 
import yfinance as yf
creds = {
    'host' : "localhost",
    'user' : "root",
     'password' : "n3u3da!",
     'database' : "CSFoundations"
}


In [13]:
res= requests.get("http://127.0.0.1:5000/api/v1/stock/AAPL/history")

In [135]:
tickers= ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'NFLX', 'FB', 'NVDA', 'DIS', 'V']
# data= []
# for ticker in tickers:
#             stock = yf.Ticker(ticker)
#             info = stock.info

#             # calculating the price growth
#             current = info.get("regularMarketPrice", 0)
#             previous = info.get("previousClose", 1) 
#             growth = (current - previous) / previous

#             # stock info
#             data.append({
#                 "ticker": ticker,
#                 "name": info.get("shortName", ""),
#                 "price": current,
#                 "previous_close": previous,
#                 "growth": growth
#             })
# data            

In [ ]:
stocks = yf.Tickers(tickers).download(period='1wk', auto_adjust=True)
day_performance = stocks.Close


[                       0%                       ]

[*********************100%***********************]  10 of 10 completed


In [ ]:
dir(yf.Tickers(tickers).download())

day_performance


[*********************100%***********************]  10 of 10 completed


Ticker,AAPL,AMZN,DIS,FB,GOOGL,MSFT,NFLX,NVDA,TSLA,V
Date,,,,,,,,,,
2025-07-31,207.570007,234.110001,119.110001,40.567001,191.899994,533.500000,1159.400024,177.869995,308.269989,345.470001
2025-08-01,202.380005,214.750000,116.589996,40.264000,189.130005,524.109985,1158.599976,173.720001,302.630005,339.350006
2025-08-04,203.350006,211.649994,119.349998,40.549999,195.039993,535.640015,1170.989990,180.000000,309.260010,342.149994
2025-08-05,202.919998,213.750000,118.320000,40.430000,194.669998,527.750000,1147.869995,178.259995,308.720001,337.429993
2025-08-06,213.250000,222.309998,115.169998,40.520000,196.089996,524.940002,1178.479980,179.419998,319.910004,339.739990
2025-08-07,219.018402,224.910004,115.050003,40.520000,196.250000,525.729980,1180.729980,183.141006,321.337006,339.820007


In [ ]:
ticker_and_quantity = pd.DataFrame(history, columns=['ticker', 'quantity', 'transaction_date'])
ticker_and_quantity['transaction_date'] = pd.to_datetime(ticker_and_quantity['transaction_date'])
holdings_df = ticker_and_quantity.sort_values(['ticker', 'transaction_date'])
holdings_df['cumulative_quantity'] = holdings_df.groupby('ticker')['quantity'].cumsum()

KeyError: 'date'

In [ ]:
from datetime import datetime as dt

def get_transaction_history( start_date: str = None, end_date: str = None) -> pd.DataFrame:
    """
    Get transaction history from database and return as DataFrame
    """
    with mysql.connector.connect(**creds) as conn:
        query = """
        SELECT ticker, stock_name, stock_value, quantity, transaction_date 
        FROM stockdemo 
        """
        params = []
        
        #Use once datepicker implemented in html
        if start_date:
            query += " AND transaction_date >= %s"
            params.append(start_date)
        if end_date:
            query += " AND transaction_date <= %s"
            params.append(end_date)
            
        query += " ORDER BY transaction_date ASC"
        
        df = pd.read_sql(query, conn, params=params)
        df['transaction_date'] = pd.to_datetime(df['transaction_date'])
        return df
        
def calculate_cumulative_holdings(transactions_df):
    """
    Calculate cumulative holdings for each ticker over time
    """
    if transactions_df.empty:
        return pd.DataFrame()
    
    # Make sure transaction_date is datetime
    # transactions_df = transactions_df.copy()
    transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])
    
    # Aggregate transactions by date and ticker (in case of multiple transactions on same day)
    daily_transactions = (transactions_df
                         .groupby(['transaction_date', 'ticker'])['quantity']
                         .sum()
                         .reset_index())
    
    # Create date range from first transaction to today
    start_date = daily_transactions['transaction_date'].min().date()
    end_date = dt.now().date()
    date_range = pd.date_range(start="2025-07-31", end=end_date, freq='D')
    
    # Get unique tickers
    tickers = daily_transactions['ticker'].unique()
    
    # Initialize result dataframe
    result = pd.DataFrame(index=date_range, columns=tickers).fillna(0.0)
    
    # For each ticker, calculate cumulative holdings
    for ticker in tickers:
        ticker_transactions = daily_transactions[daily_transactions['ticker'] == ticker].copy()
        ticker_transactions.set_index('transaction_date', inplace=True)
        
        # Reindex to match our date range
        ticker_series = ticker_transactions['quantity'].reindex(date_range, fill_value=0.0)
        
        # Calculate cumulative sum
        result[ticker] = ticker_series.cumsum()
    
    return result

In [138]:
df = get_transaction_history()
cumulative_holdings = calculate_cumulative_holdings(df)

C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_9776\2432598290.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_9776\2432598290.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = pd.DataFrame(index=date_range, columns=tickers).fillna(0.0)


In [107]:
import matplotlib.pyplot as plt
def plot_holdings(holdings_df):
    """
    Plot cumulative holdings for each ticker over time
    """
    if holdings_df.empty:
        print("No holdings data to plot.")
        return
    
    plt.figure(figsize=(14, 8))
    
    for ticker in holdings_df['ticker'].unique():
        ticker_data = holdings_df[holdings_df['ticker'] == ticker]
        plt.plot(ticker_data['transaction_date'], ticker_data['cumulative_quantity'], label=ticker)
    
    plt.title('Cumulative Holdings Over Time')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Quantity')
    plt.legend()
    plt.grid()
    plt.show()
    
plot_holdings(cumulative_holdings)

Matplotlib is building the font cache; this may take a moment.


KeyError: 'ticker'

<Figure size 1400x800 with 0 Axes>

In [139]:
cumulative_holdings 

,AAPL,AMZN,DIS,FB,GOOGL,MSFT,NFLX,NVDA,TSLA,V,AAP,AB,BSAC,CAAP
2025-07-31,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-02,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-03,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-08-05,0.0,13.0,27.25,4.0,13.3,0.0,49.0,2.6,55.0,45.0,0.0,0.0,0.0,0.0
2025-08-06,0.0,0.0,27.25,4.0,12.3,0.0,42.0,2.6,54.0,45.0,2.0,3.0,4.4,7.9
2025-08-07,21.0,12.0,32.25,9.0,27.3,1.0,82.0,82.6,63.0,46.0,2.0,3.0,4.4,7.9


In [141]:
cumulative_valuation = cumulative_holdings.multiply(day_performance, axis=0,fill_value=1).sum(axis=1)

In [134]:
day_performance

Ticker,AAPL,AMZN,DIS,FB,GOOGL,MSFT,NFLX,NVDA,TSLA,V
Date,,,,,,,,,,
2025-07-31,207.570007,234.110001,119.110001,40.567001,191.899994,533.500000,1159.400024,177.869995,308.269989,345.470001
2025-08-01,202.380005,214.750000,116.589996,40.264000,189.130005,524.109985,1158.599976,173.720001,302.630005,339.350006
2025-08-04,203.350006,211.649994,119.349998,40.549999,195.039993,535.640015,1170.989990,180.000000,309.260010,342.149994
2025-08-05,202.919998,213.750000,118.320000,40.430000,194.669998,527.750000,1147.869995,178.259995,308.720001,337.429993
2025-08-06,213.250000,222.309998,115.169998,40.520000,196.089996,524.940002,1178.479980,179.419998,319.910004,339.739990


In [142]:
cumulative_valuation

2025-07-31         0.000000
2025-08-01         0.000000
2025-08-02         0.000000
2025-08-03         0.000000
2025-08-04         0.000000
2025-08-05     97626.856453
2025-08-06     88255.760340
2025-08-07    165097.261295
dtype: float64

In [83]:
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor(dictionary=True)
    cursor.execute("select ticker, quantity, transaction_date from stockdemo order by transaction_date desc")
    history = cursor.fetchall()

In [ ]:
ticker_and_quantity = pd.DataFrame(history, columns=['ticker', 'quantity', 'transaction_date'])
ticker_and_quantity['transaction_date'] = pd.to_datetime(ticker_and_quantity['transaction_date'])
ticker_and_quantity= ticker_and_quantity.set_index('transaction_date')
ticker_and_quantity = ticker_and_quantity.sort_index()
ticker_and_quantity['quantity'] = ticker_and_quantity[ticker_and_quantity.index].groupby(['ticker'])['quantity'].cumsum()
# ticker_and_quantity['quantity'] = ticker_and_quantity['quantity'].astype(float)
# ticker_and_quantity = ticker_and_quantity.set_index('ticker')

In [50]:
#Input fake entries to stock database
with mysql.connector.connect(**creds) as stock_db:
    cursor = stock_db.cursor()
    #Create dummy entries for teh portfolio
    stockinfo = [("AAPL", "Apple", "150.20", "21"),
                    ("GOOGL", "Alphabet", "280.50", "15"),
                    ("AMZN", "Amazon", "34.00", "12"),
                    ("MSFT", "Microsoft", "2.50", "1"),
                    ("TSLA", "Tesla", "750.00", "9"),
                    ("NFLX", "Netflix", "550.00", "40"),
                    ("FB", "Facebook", "350.00", "5"),
                    ("NVDA", "Nvidia", "60.00", "80"),
                    ("DIS", "Disney", "80.00", "5"),
                    ("V", "Visa", "20.00", "1")]
    #Insert the dummy entries into the stock table
    cursor.executemany("INSERT INTO stockdemo (ticker, stock_name, stock_value, quantity) VALUES (%s, %s, %s, %s)", stockinfo)
    #Commit the changes to the database 
    stock_db.commit()